# The three levels of interface in the NAG library for Python

The [NAG Library for Python](https://www.nag.co.uk/numeric/py/nagdoc_latest/index.html) has three levels of interface available for each of our 1700+ routines.

* **base** is a pure Python Fortran-like API calling the NAG Engine. Typically, input data is type checked, some input constraints are verified, and the Engine is called. Arrays are updated in place as they would be in the Fortran Library.

* **library** is a streamlined Python wrapper that calls 'base'. This is the version that most people will want to use most of the time.  We are attempting to make it as Pythonic as possible.

* **_primitive** is a set of undocumented, direct ctypes wrappers for each routine.

##  A demonstration: Solving a linear system of equations

To demonstrate the three different interface types available, we will consider the solution of the matrix equation 

$Ax = b$

In [2]:
from numpy.random import rand
import pandas as pd
import numpy as np
import scipy.linalg
import timeit
from compare_solvers import compare_solvers #Python source code is in the same directory as this notebook

In [3]:
def generate_linear_problem(matrix_size=5,seed=2):
    """Creates example problems for Linear solvers that solve the matrix equation Ax=b"""
    np.random.seed(seed)
    A = rand(matrix_size,matrix_size)  
    b = rand(matrix_size,1)
    
    return(A,b)

In [4]:
(A,b) = generate_linear_problem(matrix_size=5,seed=2)
A

array([[0.4359949 , 0.02592623, 0.54966248, 0.43532239, 0.4203678 ],
       [0.33033482, 0.20464863, 0.61927097, 0.29965467, 0.26682728],
       [0.62113383, 0.52914209, 0.13457995, 0.51357812, 0.18443987],
       [0.78533515, 0.85397529, 0.49423684, 0.84656149, 0.07964548],
       [0.50524609, 0.0652865 , 0.42812233, 0.09653092, 0.12715997]])

In [5]:
b

array([[0.59674531],
       [0.226012  ],
       [0.10694568],
       [0.22030621],
       [0.34982629]])

We have $A$ and $b$ so our task is to find $x$.  For this we will use the NAG routine **dgesv** which is taken from LAPACK

# naginterfaces.library - easy to use and Pythonic

In [7]:
#Generate problem
(A,b) = generate_linear_problem(matrix_size = 5,seed=2)

# Solve with the easiest to use version of NAG's general linear solver
from naginterfaces.library.lapacklin import dgesv
[a,ipiv,x] = dgesv(A, b)

In [7]:
# As you might expect, the input matrices are unchanged.  Here is A
A

array([[0.4359949 , 0.02592623, 0.54966248, 0.43532239, 0.4203678 ],
       [0.33033482, 0.20464863, 0.61927097, 0.29965467, 0.26682728],
       [0.62113383, 0.52914209, 0.13457995, 0.51357812, 0.18443987],
       [0.78533515, 0.85397529, 0.49423684, 0.84656149, 0.07964548],
       [0.50524609, 0.0652865 , 0.42812233, 0.09653092, 0.12715997]])

In [8]:
# And here is b
b

array([[0.59674531],
       [0.226012  ],
       [0.10694568],
       [0.22030621],
       [0.34982629]])

In [9]:
# The solution is in it's own vector
x

array([[ 0.62236818],
       [-1.41921342],
       [ 0.2098716 ],
       [ 1.03787095],
       [-0.48761155]])

In [9]:
#Demonstrate that the matrix product Ax = b
A @ x

array([[0.59674531],
       [0.226012  ],
       [0.10694568],
       [0.22030621],
       [0.34982629]])

In [8]:
#There is documentation for every routine in naginterfaces.library
?dgesv

In [9]:
#You can see the source code for the wrapper
??dgesv

# naginterfaces.base - More like Fortran

In [12]:
from naginterfaces.base.lapacklin import dgesv as dgesv_base
#Looks more like a Fortran routine in that it overwrites the input matrices 
#A litttle harder to use and requires more set up by the user but can be faster

#Generate problem
(A,b) = generate_linear_problem(matrix_size = 5,seed = 2)

#Have to define more input variables than the library version
N=5  # In the the Fortran-like interface, matrices don't know how big they are
ipiv = np.zeros(N,dtype='int64');

#Note that there is no explicit output. The matrices in the input arguments are overwritten 
dgesv_base(N,1,A,ipiv,b)

For example, A is no longer the original matrix A, it now contains the the factors L and U from the factorization A = PLU; the unit diagonal elements of L are not stored.

In [13]:
A

array([[ 0.78533515,  0.85397529,  0.49423684,  0.84656149,  0.07964548],
       [ 0.64335092, -0.48411929,  0.1101546 , -0.4481052 ,  0.07591998],
       [ 0.42062911,  0.3192565 ,  0.37621299,  0.08662677,  0.20908812],
       [ 0.55517049,  0.92575459,  0.46064501,  0.34026769,  0.20955231],
       [ 0.79091562,  0.30215756, -0.76978663,  0.13548722,  0.2310688 ]])

Similarly, the original input vector `b` has been overwritten with the solution `x`

In [14]:
b

array([[ 0.62236818],
       [-1.41921342],
       [ 0.2098716 ],
       [ 1.03787095],
       [-0.48761155]])

We can already imagine that this might be more efficient with respect to memory since we are reusing memory wherever possible rather than allocating new memory for solutions.

We will see what this might do for us later.

the `naginterfaces.base` versions of all functions are also fully documented

In [15]:
?dgesv_base

In [16]:
??dgesv_base

# naginterfaces._primitive - close to the metal
Primitive, and difficult to use but possibly useful.  These are the ctypes headers to the compiled library functions.

There is no documentation.

All you have is the function definitions themselves at `/site-packages/naginterfaces/_primitive/`.  For example, The `_primitive` version of dgesv is in `/site-packages/naginterfaces/_primitive/lapacklin.py` and contains the following

```
dgesv = utils._EngineState._get_symbol('dgesv')
dgesv.restype = None
dgesv.argtypes = (
    utils._BYREF_INT, # Input, n.
    utils._BYREF_INT, # Input, nrhs.
    utils._EngineFloat64ArrayType.c_type, # Input/Output, a[lda, *].
    utils._BYREF_INT, # Input, lda.
    utils._EngineIntArrayType.c_type, # Output, ipiv[n].
    utils._EngineFloat64ArrayType.c_type, # Input/Output, b[ldb, *].
    utils._BYREF_INT, # Input, ldb.
    utils._BYREF_INT, # Output, info.
)
f07aaf = utils._EngineState._get_symbol('f07aaf')
f07aaf.restype = dgesv.restype
f07aaf.argtypes = dgesv.argtypes
```

Here's how you use it

In [17]:
from naginterfaces._primitive.lapacklin import dgesv as dgesv_primitive
from naginterfaces.base import utils
import ctypes
#Close to the metal function that requires knowledge of ctypes, extra setup and care that you don't shoot off your own foot

#Generate problem
(A,b) = generate_linear_problem(matrix_size = 5,seed=2)

#Create ctypes input and output arguments
N_c = utils.EngineIntCType(N)
one_c = utils.EngineIntCType(1)
A_c = A.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
ipiv_c = ipiv.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
b_c = b.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
info = ctypes.c_int(0);

dgesv_primitive(N_c, one_c ,A_c ,N_c ,ipiv_c, b_c, N_c,info)

The solution is not directly viewable since it's just a pointer:

In [18]:
b_c

We need to convert this pointer to a numpy array if we are to view the results.  

In [19]:
np.ctypeslib.as_array(b_c,(N,1))

array([[-0.07647678],
       [ 0.37618264],
       [ 1.70266994],
       [-0.90186141],
       [ 0.3097503 ]])

There may be situations, however, where we might prefer to omit the final step. For example, if we want to pipe the result into additional _primitive functions.

# Speed differences between interface levels

The natural qestion that arises when considering these different interfaces regards speed. One may expect that the closer you are to the pure, compiled Fortran code, the faster you'll get the result.  In the case of **dgesv** this turns out to be true  

In [23]:
#library interface - Easy to use
from pytictoc import TicToc
timer = TicToc()
matrix_size=12000
(A,b) = generate_linear_problem(matrix_size,seed=2)
timer.tic()
[a,ipiv,x] = dgesv(A, b)
timer.toc()

Elapsed time is 10.998698 seconds.


In [24]:
#base interface - Fortran-like API
(A,b) = generate_linear_problem(matrix_size,seed=2)
#Have to define more input variables for this version of the API
N=matrix_size
ipiv = np.zeros(N,dtype='int64');

timer.tic()
dgesv_base(N,1,A,ipiv,b)
timer.toc()

Elapsed time is 10.422995 seconds.


In [25]:
#_primitive interface - ugly but fast
(A,b) = generate_linear_problem(matrix_size,seed=2)

N_c = utils.EngineIntCType(N)
one_c = utils.EngineIntCType(1)
A_c = A.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
ipiv_c = ipiv.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
b_c = b.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
info = ctypes.c_int(0);

timer.tic()
dgesv_primitive(N_c, one_c ,A_c ,N_c ,ipiv_c, b_c, N_c,info);
timer.toc()

Elapsed time is 9.022218 seconds.


In the above run, there is almost 20% difference between the fastest and slowest versions of the interface

# Another trick for speed - Fortran arrays

Numpy arrays have C-type storage by default.  That is, the **rows** are contiguous in memory.  We can see this with the **flags** attribute

In [26]:
mymat = np.zeros((2,2))
mymat.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

LAPACK routines such as **dgesv** require that input matrices are stored with Fortran-type storage where the **columns** are contiguous in memory.  If they get the wrong type of matrix, they need to perform a potentially costly transpose step.  

We can convert numpy arrays to follow the Fortran convetion as follows

In [27]:
mymat = np.asfortranarray(mymat)
mymat.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

This conversion takes time of course but it may be worth it if your algorithm contains many matrix operations.

In [28]:
matrix_size=12000
#Create C-ordered problem
(A_c,b_c) = generate_linear_problem(matrix_size,seed=2)
# Create Fortran ordered versions
A_f = np.asfortranarray(A_c);
b_f = np.asfortranarray(b_c);

In [29]:
from pytictoc import TicToc
timer = TicToc()

timer.tic()
[a,ipiv,x] = dgesv(A_c, b_c) # C-ordered input matrices
timer.toc()

Elapsed time is 11.171874 seconds.


In [30]:
timer.tic()
[a,ipiv,x] = dgesv(A_f, b_f) # Fortran-ordered matrices
timer.toc()

Elapsed time is 8.956717 seconds.


# NAG has all of LAPACK - Mixed precision solvers

Along with its own algorithmic content, The NAG library for Python gives us access to all of LAPACK including **dsgesv** which uses single precision internally whenever it can while aiming for full double precision in the output.
This can be significantly faster in some cases.

NAG is currently actively researching new mixed precision routines in collaboration with the University of Manchester 

In [34]:
from naginterfaces.library.lapacklin import dsgesv
(A,b) = generate_linear_problem(matrix_size,seed=2)
# Chnage to Fortran ordering
A = np.asfortranarray(A);
b = np.asfortranarray(b);

timer.tic()
[a,ipiv,x,itera] = dsgesv(A, b)
timer.toc()

Elapsed time is 5.870411 seconds.


We also have access to the **base** and **_primitive** interfaces to **dsgesv** so we can try those out too.

In [35]:
from naginterfaces.base.lapacklin import dsgesv as dsgesv_base
(A,b) = generate_linear_problem(matrix_size,seed=2)
# Chnage to Fortran ordering
A = np.asfortranarray(A);
b = np.asfortranarray(b);

N=matrix_size
ipiv = np.zeros(N,dtype='int64');
x = np.zeros((N,1))

timer.tic()
dsgesv_base(N, 1, A, ipiv, b, x)
timer.toc()

Elapsed time is 5.148290 seconds.


In [36]:
from naginterfaces._primitive.lapacklin import dsgesv as dsgesv_primitive
from naginterfaces.base import utils
import ctypes

(A,b) = generate_linear_problem(matrix_size,seed=2)
# Chnage to Fortran ordering
A = np.asfortranarray(A);
b = np.asfortranarray(b);

N = matrix_size
x = np.zeros((N,1))
ipiv = np.zeros(N,dtype='int64');
work = np.zeros((matrix_size,1))
swork = np.zeros((matrix_size*(matrix_size+1),1),dtype=np.float32)
work = np.asfortranarray(work);
swork = np.asfortranarray(swork);

N_c = utils.EngineIntCType(N)
one_c = utils.EngineIntCType(1)
A_c = A.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
ipiv_c = ipiv.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
b_c = b.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
x_c = x.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
work_c = work.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
swork_c = swork.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
info = ctypes.c_int(0);
itera = ctypes.c_int(0);

timer.tic()
dsgesv_primitive(N_c, one_c ,A_c ,N_c ,ipiv_c, b_c, N_c,x_c,N_c,work_c,swork_c,itera,info);
timer.toc()

Elapsed time is 5.247763 seconds.


In the above, you'll note that the **_primitive** interface is slightly slower than the **base**.  This is because the overhead in **base** is so small for this routine that what we seeing here is the natural small variation between individual runs.  

# More robust timings

The timings that we have demonstrated above are not very robust since many factors can affect a single computation. For example, a laptop CPU may only be able to maintain its fastest clock speed for a short time before it has to slow down for thermal reasons.  There may be other programs running or operating system tasks happening in the background that affect individual timings.

In an attempt to control for this kind of thing, there is a `compare_solvers` routine in the same directory as this notebook that uses Python's timieit module for more robust timings.

Below, we run each computation 10 times and report the average result in seconds.

In [3]:
result = [compare_solvers(mat_size,number=10,fortran_order=False,res_form='raw') for mat_size in [10000]]
result = pd.DataFrame(result)
result.columns=["MatSize","Scipy","NAG:dgesv","NAG:dgesv_base","NAG:dgesv_primitive","NAG:dsgesv","NAG:dsgesv_base"]
result.set_index('MatSize')
result

,MatSize,Scipy,NAG:dgesv,NAG:dgesv_base,NAG:dgesv_primitive,NAG:dsgesv,NAG:dsgesv_base
0,10000.0,8.672804,8.565575,8.277488,7.011921,5.623696,4.727431


Now we try the same again with Fortran-ordered matrices.

In [4]:
result = [compare_solvers(10000,number=10,fortran_order=True,res_form='raw')]
result = pd.DataFrame(result)
result.columns=["MatSize","Scipy","NAG:dgesv","NAG:dgesv_base","NAG:dgesv_primitive","NAG:dsgesv","NAG:dsgesv_base"]
result.set_index('MatSize')
result

,MatSize,Scipy,NAG:dgesv,NAG:dgesv_base,NAG:dgesv_primitive,NAG:dsgesv,NAG:dsgesv_base
0,10000.0,8.188727,7.889164,7.17841,7.992964,4.273695,4.641136


# Making use of Matrix structure

The NAG library often contains specialised varations for any given mathematical problem.  In the case of linear solvers, one set of variations caters for coefficient matrices that have a certain structure.  When your problem exhibits such structure, it often pays divideds to use the specialised solver.

## Solving linear systems of Toeplitz matrices - An example of exploiting stucture

Toeplitz matrices occur in various applications. A couple of examples are given at http://jack.valmadre.net/notes/2015/03/28/symmetric-positive-toeplitz/

Here we solve a linear system where the coefficient matrix is a Toeplitz matrix.  First, we make no use of the underlying structure and just use NAG's **dgesv**

In [22]:
from naginterfaces.library.lapacklin import dgesv   # A general solver
from naginterfaces.library.linsys import real_toeplitz_solve # A toeplitz solver
from pytictoc import TicToc
import numpy as np
import scipy.linalg

timer = TicToc()

# Construct a real, symmetric, positive definite toeplitz matrix 
matrix_size = 5000
t = np.arange(0,matrix_size);
a = np.exp(-np.abs(t)/10);
# The toeplitz matrix is defined by it's diagonals.  We can construct the full matrix from the diagonals using scipy
A = scipy.linalg.toeplitz(a, a)
# Construct and a random Righ hand side
np.random.seed(2)
b = np.random.rand(matrix_size,1)

timer.tic()
[asol,ipiv,x_nag_gen] = dgesv(A, b)
timer.toc()

Elapsed time is 12.231292 seconds.


We will now solve the same problem but make use of the function **real_toeplitz_solve**

In [23]:
# NAG's toeplitz solver requires that b be of dimension N instead of N,1
# So we reconstruct it with the correct dimension
np.random.seed(2)
b = np.random.rand(matrix_size)

timer.tic()
[x_nag_toe,p] = real_toeplitz_solve(matrix_size,a,b,wantp=False)
timer.toc()

Elapsed time is 0.068450 seconds.


In [24]:
# Let's see how many times faster it is to use the toeplitz solver
4.96/0.039

127.17948717948718

It is **well over 100x faster** to use the Toeplitz solver.  We should check that we get the same answers?  
Annoyingly, the solution matrices are are different shapes!

In [20]:
#Solution from the general solver
x_nag_gen.shape

(5000, 1)

In [21]:
#Solution from the toeplitz solver
x_nag_toe.shape

(5000,)

but the values agree to high accuracy

In [112]:
np.max(abs(x_nag_gen - np.reshape(x_nag_toe,(matrix_size,1))))

1.1191048088221578e-13

## Other linear solvers in the NAG library for Python that make use of various matrix structures 

Your application may not be able to make use of Toeplitz solvers but it may well be able to take advantage of one of the other specialised linear solvers in the NAG library.  Here's the full list